## ¿Qué es el Aprendizaje por Refuerzo?


Seguramente ya conocerás las 2 grandes áreas de aprendizaje tradicional del Machine Learning, el aprendizaje supervisado y el aprendizaje no supervisado. Parece difícil que aquí hubiera espacio para otras opciones; sin embargo sí la hay y es el Aprendizaje por refuerzo. En aprendizaje por refuerzo (ó Reinforcement Learning en inglés) no tenemos una “etiqueta de salida”, por lo que no es de tipo supervisado y si bien estos algoritmos aprenden por sí mismos, tampoco son de tipo no supervisado, en donde se intenta clasificar grupos teniendo en cuenta alguna distancia entre muestras.

![foto1](img/areas-ml.png)

![foto4](img/1_YIETknPBlQQBF40DJxL8QA.png)

Si nos ponemos a pensar, los problemas de ML supervisados y no supervisados son específicos de un caso de negocio en particular, sea de clasificación ó predicción, están muy delimitados, por ejemplo, clasificar “perros ó gatos“, ó agrupar “k=5” clusters. En contraste, en el mundo real contamos con múltiples variables que por lo general se interrelacionan y que dependen de otros casos de negocio y dan lugar a escenarios más grandes en donde tomar decisiones. Para conducir un coche no basta una inteligencia que pueda detectar un semáforo en rojo, verde ó amarillo; tendremos muchísimos factores -todos a la vez- a los que prestar atención: a qué velocidad vamos, estamos ante una curva?, hay peatones?, es de noche y debemos encender las luces?.

Una solución sería tener múltiples máquinas de ML supervisadas y que interactúan entre si -y esto no estaría mal- ó podemos cambiar el enfoque… Y ahí aparece el Reinforcement Learning (RL) como una alternativa, tal vez de las más ambiciosas en las que se intenta integrar el Machine Learning en el mundo real, sobre todo aplicado a robots y maquinaria industrial.

El Reinforcement Learning entonces, intentará hacer aprender a la máquina basándose en un esquema de “premios y castigos” -cómo con el perro de Pablov- en un entorno en donde hay que tomar acciones y que está afectado por múltiples variables que cambian con el tiempo.

![foto2](img/496302-1548589.jpg)


## Diferencias con “los clásicos”

En los modelos de Aprendizaje Supervisado (o no supervisado) como redes neuronales, árboles, knn, etc, se intenta “minimizar la función coste”, reducir el error.

En cambio en el RL se intenta “maximizar la recompensa“. Y esto puede ser, a pesar de a veces cometer errores ó de no ser óptimos.

Componentes del RL
El Reinforcement Learning propone un nuevo enfoque para hacer que nuestra máquina aprenda, para ello, postula los siguientes 2 componentes:

el Agente: será nuestro modelo que queremos entrenar y que aprenda a tomar decisiones.

Ambiente: será el entorno en donde interactúa y “se mueve” el agente. El ambiente contiene las limitaciones y reglas posibles a cada momento.
Entre ellos hay una relación que se retroalimenta y cuenta con los siguientes nexos:

Acción: las posibles acciones que puede tomar en un momento determinado el Agente.

Estado (del ambiente): son los indicadores del ambiente de cómo están los diversos elementos que lo componen en ese momento.

Recompensas (ó castigos!): a raíz de cada acción tomada por el Agente, podremos obtener un premio ó una penalización que orientarán al Agente en si lo está haciendo bien ó mal.

![foto3](img/premio-castigo.png)



## Empecemos a trabajar Obreros..

In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings
warnings.filterwarnings("ignore")

### Cart Pole.

![foto5](img/0-923627-95182.gif)

Se trata de un péndulo que está en posición vertical y debemos mover el carro de abajo para que no caiga a ninguno de los lados.

Este entorno es un problema clásico y como tal, ya está diseñado por nosotros en diferentes librerías. Nosotros usaremos OpenAI Gym. Se trata de una librería que provee de varios entornos interesantes para poner a prueba nuestros algoritmos, con representación gráfica de los mismos.

Este entorno se caracteriza por tener cuatro entradas: posición del carro, velocidad del carro, ángulo del péndulo y velocidad angular del péndulo. Y dos acciones: acelerar a la izquierda y acelerar a la derecha. Las recompensas son 1 por cada acción tomada. El episodio finaliza cuando el ángulo del péndulo supera los 12 grados, el carro se aleja demasiado del centro y cuando el episodio alcanza los 200 pasos.

Vamos a ver como funciona la interfaz Gym para que luego seamos capaces de crear nuestros propios entornos. Esta se compone de:

Una clase que hereda de **gym.Env**

Un constructor donde inicializamos, como mínimo, self.action_space, self.observation_space y opcionalmente self.reward_range y parámetros internos del entorno nuestro. Estas dos primeras deben ser de algún subtipo **gym.spaces**. Los más normales son **Discrete**, que es básicamente un valor activado sobre N posibles y Box, que es una matriz de características (podemos especificar la forma para que sea unidimensional, o que tenga más dimensiones), que contiene números. 
Es muy habitual que las observaciones sean de tipo Box, ya que se miden varias características a la vez, con números decimales y **Discrete** sea para las acciones ya que tomamos una acción de N posibles. Pero otras combinaciones son posibles y existen más tipos de **gym.spaces**. Si vamos a introducir características diferentes dentro del mismo Box es muy conveniente normalizar los datos entre -1 y 1. self.reward_range, al contrario, es una tupla en la que se especifica el valor máximo y el mínimo que pueden alcanzar las recompensas.

Un método **reset**, que reinicia el entorno y devuelve una observación inicial. Estas observaciones, deberán devolverse con NumPy si son de tipo Box y coincidir con lo declarado en el constructor en cuanto a forma y tipos.

Un método **step** que toma una acción y devuelve una tupla con cuatro valores: nueva observación, recompensa, un boolean de si el episodio ha acabado ya o no y un diccionario de información extra (no usado por los algoritmos).

Opcionalmente pueden llevar instrucciones sobre como renderizar el entorno y así ver en vivo o en vídeo el funcionamiento de los algoritmos. Principalmente se controla a través del método **render**.

[cartpole](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py)

In [2]:
env=gym.make('CartPole-v0') # Crea el entorno de juego correspondiente

env.seed(1) #Opcional, establezca un número aleatorio para que el proceso se pueda repetir

env=env.unwrapped #Opcional, agregue restricciones al medio ambiente, beneficioso para la capacitación

# ---------------------- Espacio de acción y espacio de estado --------------------- #

print("Action Space {}".format(env.action_space)) # Posibles Movimientos

print("State Space {}".format(env.observation_space))

Action Space Discrete(2)
State Space Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [3]:
env = gym.make('CartPole-v0')

In [4]:
env.reset()
for _ in range(100):
    env.render()
    env.step(env.action_space.sample())

In [5]:
env.close()

Bien ahora tenemos un carrito idiota que no hace nada solo moverse...

Si queremos tomar acciones que son mejores que las aleatorias en cada paso, entonces sería bueno comprender realmente cómo nuestras acciones afectan el medio ambiente. A
La función de paso del entorno devolverá la información que necesitamos. La función paso devuelve cuatro valores, la siguiente es la información específica:

Observación (objet): un objeto relacionado con el entorno describe el entorno que observa, como la información de píxeles de la cámara, el ángulo y la velocidad angular del robot y el estado del tablero en un juego de mesa.

recompensa (float): La suma de todas las recompensas obtenidas de acciones anteriores. Los diferentes entornos tienen diferentes métodos de cálculo, pero el objetivo siempre ha sido aumentar tus recompensas totales.

done (booleano): para determinar si es el momento de restablecer el entorno, la mayoría de las tareas se dividen en episodios claramente definidos, y done es True para indicar que el episodio ha terminado.

info (dict): información de diagnóstico para depuración. A veces también es útil para aprender (por ejemplo, puede contener la probabilidad original después del último cambio de estado del entorno). Pero la evaluación formal del agente no permite el uso de esta información para el aprendizaje

[cartpole.py](./cartpole.py)

In [7]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample() ## modificar!!!
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

[-0.03494446  0.02992036  0.0361635   0.02328484]
[-0.03434605 -0.16570105  0.03662919  0.32715484]
[-0.03766007 -0.36132482  0.04317229  0.63116026]
[-0.04488657 -0.55702174  0.05579549  0.9371208 ]
[-0.056027   -0.7528499   0.07453791  1.2468013 ]
[-0.071084   -0.55875874  0.09947394  0.97836685]
[-0.08225918 -0.75506353  0.11904127  1.3005655 ]
[-0.09736045 -0.95147794  0.14505258  1.6280175 ]
[-0.11639    -0.7583287   0.17761293  1.3838323 ]
[-0.13155659 -0.5658104   0.20528959  1.151542  ]
Episode finished after 10 timesteps
[ 0.02402956  0.00345985 -0.00604836  0.03795605]
[ 0.02409875 -0.19157486 -0.00528924  0.32872453]
[ 0.02026726 -0.38662112  0.00128525  0.6197348 ]
[ 0.01253483 -0.581761    0.01367994  0.91282225]
[ 0.00089961 -0.38682675  0.03193639  0.62447   ]
[-0.00683692 -0.58237964  0.04442579  0.92703754]
[-0.01848451 -0.38788486  0.06296654  0.64864004]
[-0.02624221 -0.5838248   0.07593934  0.9604675 ]
[-0.03791871 -0.38980123  0.09514869  0.6925756 ]
[-0.04571473 -

## Stable Baselines

Este algoritmo, DQN, es muy popular, y se encuentra implementado en diferentes librerías. Una de las más interesantes es Stable Baselines 3, que intenta ser el sklearn del aprendizaje por refuerzo. Usa PyTorch internamente. Con Stable Baselines 3, debemos proporcionar un entorno que siga la interfaz Gym y ajustar los hiperparámetros. En el caso de DQN los hiperparámetros más importantes son:

- **policy** - La política a usar del modelo. Casi siempre MlpPolicy. Si la entrada son imágenes, CnnPolicy.
- **env** - El entorno sobre el que vamos a aprender. Debe implementar la interfaz OpenAI Gym
- **learning_rate** - Ratio de aprendizaje de la red neuronal
- **buffer_size** - El tamaño del buffer que almacenará las transiciones del "experience replay".
- **batch_size** - Tamaño del batch que se usa para reentrenar.
- **learning_starts** - Cuantos steps debe dar el modelo antes de empezar a aprender la red neuronal.
- **gamma** - el factor de descuento. En posts anteriores hemos hablado de él.
- **train_freq** - Cada cuantos steps se reentrena el modelo.
- **gradient_steps** - Cuantos pasos de gradiente se dan al entrenar. Por defecto, 1.
- **target_update_interval** - Cada cuantos steps se actualiza el "fixed Q-target".
- **policy_kwargs** - Ajustes de la política. En el caso de MlpPolicy podremos ajustar la forma de la red, 
                    así como las funciones de activación   y más detalles.

In [ ]:
from stable_baselines3 import DQN

In [ ]:
env = gym.make("CartPole-v1")
env.reset()
for _ in range(100):
    env.render()
    env.step(env.action_space.sample())

In [ ]:
env.close()

In [ ]:
model = DQN(
    "MlpPolicy",
    env,
    learning_rate=1e-3,
    buffer_size=50000,
    learning_starts=10000,
    batch_size=64,
    gamma=0.999,
    gradient_steps=1,
    train_freq=20,
    target_update_interval=2000,
    verbose=1
    )
model.learn(total_timesteps=500_000)
model.save("dqn_cartpole")

In [ ]:
model = DQN.load("dqn_cartpole")

obs = env.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()

In [ ]:
env.close()

Ahora vamos con otro ejemplo..

## MountainCar

![foto7](img/MountainCar.gif)

In [ ]:
env = gym.make('MountainCar-v0')

In [ ]:
env.reset()
for i in range(300):
    env.render()
    env.step(env.action_space.sample()) # accion random


In [ ]:
env.close()

In [ ]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

[mountaincar.py](./mountaincar.py)

Otro ejemplo..

## Taxi

![foto8](img/ezgif.com-video-to-gif1.gif)

In [ ]:
env = gym.make("Taxi-v3").env
env.render()

Hay 4 ubicaciones (etiquetadas con letras diferentes), y nuestro trabajo es recoger al pasajero en una ubicación y dejarlo en otra. Recibimos +20 puntos por una entrega exitosa y perdemos 1 punto por cada paso de tiempo que toma. También hay una penalización de 10 puntos por acciones ilegales de recogida y entrega.

In [ ]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))


In [ ]:
print(env.s)


Como se verifica en las impresiones, tenemos un espacio de acción de tamaño 6 y un espacio de estado de tamaño 500. Como verá, nuestro algoritmo RL no necesitará más información que estas dos cosas. Todo lo que necesitamos es una forma de identificar un estado de manera única asignando un número único a cada estado posible, y RL aprende a elegir un número de acción del 0 al 5 donde:

- 0 = sur
- 1 = norte
- 2 = este
- 3 = oeste
- 4 = recoger
- 5 = dejar

In [ ]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

In [ ]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    # Añade +1 a la variable penaltie cuando el reward sea -10
    if reward == -10:
        penalties = penalties + 1
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

In [ ]:
# Crea un array2d de ceros del tamaño de los diferentes estados y las diferentes posibles acciones
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

In [ ]:
random.uniform(0, 1)

In [ ]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    env.reset()
    # Crea el estado inicial
    state = env.encode(3, 1, 2, 0)
    env.s = state
    # Inicializa las epochs, penalties y rewards
    epochs, penalties, reward = 0, 0, 0

    done = False
    actions = []
    while not done:
        # Elige la accion que te indique el maximo valor de la q_table
        action = np.argmax(q_table[state])
        actions.append(action)
        # Ejecuta la accion
        state, reward, done, info = env.step(action)

        # Actualiza el valor de penalties si el reward es -10
        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

In [ ]:
actions

In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    env.reset()
    # Crea el estado inicial
    state = env.encode(3, 1, 2, 0)
    env.s = state
    # Inicializa las epochs, penalties y rewards
    epochs, penalties, reward = 0, 0, 0

    done = False
    actions = []
    while not done:
        # Elige la accion que te indique el maximo valor de la q_table
        action = env.action_space.sample()
        actions.append(action)
        # Ejecuta la accion
        state, reward, done, info = env.step(action)

        # Actualiza el valor de penalties si el reward es -10
        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

ahora vamos con algo mas dificil

## Mario

![foto9](img/DDQN-1.gif)

Homer le asignamos una tarea?

# ...

![foto10](img/homero-homer.gif)

## FrozenLake

In [ ]:
# crear e inicializar el env
env = gym.make('FrozenLake-v1', is_slippery=False)

In [ ]:
env.reset()

In [ ]:
env.render()

In [ ]:
is_done = False
t = 0
while not is_done:
    action = env.action_space.sample()
    state, reward, is_done, _ = env.step(action)
    env.render()
    t += 1
print('\nlast state =', state)
print('reward = ', reward)
print('time steps =', t)

In [ ]:
state, reward, is_done, _ = env.step(action)